<a href="https://colab.research.google.com/github/lustraka/Data_Analysis_Workouts/blob/main/Analyse_Twitter_Data/wrangle_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reporting: wragle_report
* Create a **300-600 word written report** called "wrangle_report.pdf" or "wrangle_report.html" that briefly describes your wrangling efforts. This is to be framed as an internal document.

## Gather data
![Gather Data Diagram](http://www.plantuml.com/plantuml/png/ZP7DIiD04CVl-nH33ZOUsfDua1egYE3D1n32a9sq8yacxapwWVZkxYOf2PP2Bu7_x7pyVcGZYbwxkZ9OLSsXvYExhJ4bLmI97bYLV8w-sF6USd8xT0NPLI7x153WMeCXfwmdi2N2UlsAYcyUQyeo-35w8DASBJm1EtryCB_csXfOBZD8FA7DmyNqFeaBZIcQkcqSEGLFtns9oiZBDMufRprPBfGR9oiTGIzU9aA-XV1y298jQThvAh14kCkz5vL_4aBjy-T25xqCpsNYsVOKQ5u-thDiL1fOlgHnAprgxpALPiCnJavwYzDuBnhbegYTfEUkqIwnNTrA6bA_FdvW7Uf3srHBgWQjfEUfqZTr7lM7FwOcMB8leWwAa0ZSGKZeUfsXP8mjCOwFEKIUHGXPvyyRSW6k-WC0)

## Clean Data
![Data Cleaning](http://www.plantuml.com/plantuml/png/pLHVQzim47_Nfn1z62saXIQb3oEOmie-mRQNCOnYdJqL_J7IgKqO-Uuxso5IOeCtmaRVV7S_t_r-fxjx2YWyHgEPqDgz6RLLjc5WKU6EorhmzVfPeFZiXO6YuByOlW5WgIJ_o9N5nmUcREcXrRiINKd9C8OXMcidpkCnW3-DeZ90VvcMV6xp0XMouAAl8Da8MqoMXvu1oAbYu-YeuXWoGG6shR2I8D7haDKMIDqfRJH0dyuVUt08IB3EA7iIii9aMgHh6fTCukHgyCdHnhOzjpFyFhsCcd-PbxRf1AJVbUd97m7jVLRflgoShRD6aOVeacFZd6RjCiCkHzZb2NQLOLSZx2h7Pf9mHb9wF4ePBHwzjNXrNY6SkVUCLLg4mAt3aW2DiaANVO90fVZryyA3lE5TgGl7-BzCvPvr2tbxMqm9whIwAFdL-WDw4NuMLrm4td-oAVrtTyKXUcWIvIvVXG-Tm4g3iDIYws-B7NHtuQRlqPZJvIkHL8qPJ553BsoedJS3FKTc0mEzIvThJ2IlbyYeFo7pFdQBo7QAfacpj7f3l-VtBvy0AH0w-_7XkmqB_lNBfn_Jj_lVdrMYr2Hp_l1jF81CuR-yAkDD6Ley66nwMdC8vUeP4rJDGtko7VIy39uwdi3AQFHl)